# Work Extraction and Deduplication (also useful for "dublettenkontrolle")

The problem:

1. We want to check if in the instances we have as our STAR records, there are any that are the same FRBR work. That is, has the same intellectual content, with different STAR records holding different Instances/Manifestations of that work. An example would be an article that has appeared as a chapter in both a Springer living reference (electronic only) and as a chapter in the final, finished Springer reference book (book can be published in electronic forms - pdf and epub - and in print (both hardcover and softcover are possible)).

At first glance, simply checking titles for equality would seem to be a good idea. However, sometimes we have articles that have the same title (and author(s)), but that appear every year (in the same journal) with updated or completely new content. These would be new works. A good example are the articles titled [Aktuelle Ergebnisse der Werbewirkungsforschung](https://pubpsych.zpid.de/pubpsych/Search.action?sortBy=score&dir=desc&q=TI%3D%22Aktuelle+Ergebnisse+der+Werbewirkungsforschung%22) by Uli Gleich. There are 14 records with that title and author, each a different work and with a different publication year. They all appear in the "Media Perspektiven" journal (ISSN: 0170-1754). 

What can we do to make sure that these will not be false positives when doing automatic work extraction and merging?

How are they the same:
- same title
- same author(s)
- same journal
- same journal issn (usually, sometimes there is no issn)
- same MT (Print) - usually, sometimes there is no MT
- same language
- no volume and issue information for the journal in all of them 

How do they differ?
- a different publication year - but that could happen for records that have the same work, as well (though they usually appear in a short timespan, like max 2 years apart?)
- abstracts: they are different, sometimes only a little. We could try a fuzzywuzzy comparison or compare a hash.
- **14 records**: maybe we could use the fact that there is **more than the "usual" number of "duplicates"**. We usually only have a few records which could be the same work - like two or maybe three. But here we have 14. That's a lot. Maybe we could use that as a heuristic to say: if there are more than 5 records with the same title and author, then we should check them manually or just don't merge them at all.
- they _usually_ have **different number of "Literaturangaben"** - references? That's work-level info. But that's mere coincidence, it might be the same number and still not be the same work. It might serve as a secondary heuristic, though.
- the pages are different (but that may not always the case; also: that would only be different for different instances, not works)

We might end up just excluding these records from the work extraction process by title or key. Especially if we already know that we don't have real record duplicates (double instances) for them.

If we can be sure there are no real record duplicates in PSYNDEX, then we can use instance info to check if they are the same work. For example, if they are all the same media type (MT=Print) and the same journal (title), and we don't have duplicates in the database, we can be reasonably sure that they are NOT the same work, since they apparently are all in the same format and journal, so published as a kind of running series of yearly articles. 


## What we wish to merge as the same work - examples:

### Chapter that first appears digitally in a Springer Living Reference, then in different formats in the finished reference with the same book title
(Interkulturelle Kompetenz: Kritische Betrachtung eines Konstrukts)[https://pubpsych.zpid.de/pubpsych/Search.action?sortBy=PY_sort&dir=desc&q=DFK%3D%280368936+or+0360687] by Petia Genkova. Both are chapters (DT=05)
- One was published in 2019 in a book with DFK 0348273.
- The other in 2020 in a book with DFK 0365952. 
- The **books they are in** [Handbuch Migration und Erfolg. Psychologische und sozialwissenschaftliche Aspekte](https://pubpsych.zpid.de/pubpsych/Search.action?sortBy=PY_sort&dir=desc&q=DFK%3D%280348273+or+0365952) both have the same title, publisher and publication place, but **different DFKs and different ISBNs** (and different publication years and DOIs). 
- <BIP>Handbuch Migration und Erfolg. Psychologische und sozialwissenschaftliche Aspekte</BIP><SSPU>|v Springer Fachmedien |o Wiesbaden |e 978-3-658-18403-2</SSPU>
    - only this one has series info that indicates it is a living reference:  <SSSE>Springer Reference Psychologie - Living reference work</SSSE>
- <BIP>Handbuch Migration und Erfolg. Psychologische und sozialwissenschaftliche Aspekte</BIP><SSPU>|v Springer Fachmedien |o Wiesbaden |i 978-3-658-18235-9 |e 978-3-658-18236-6</SSPU>
The chapters have slightlly **different DOIs**: 10.1007/978-3-658-**18236-6_38** vs 10.1007/978-3-658-**18403-2_38-1**

- what is the same to indicate this can be merged into the same work with two instance bundles (and two instances in one bundle, the print and online version)?
    - title (and possible subtitle)
    - full author list
    - the abstract is "pretty much" the same. We could try a fuzzywuzzy comparison or a hash. 
    - only 2 records with the same title and authors
    - published within 1 year of each other
    - book title (that instance level info, though)
    - they are definitely different instances (not record/instance duplicates): chapter DOI, container identifiers (isbns, DOIs, DFKs)

### A thesis that is indexed as a book/monograph and once as a (gray lit) thesis
Title: Ein Veränderungsmodell prosozialer Handlungen: Leistungen erwachsener Töchter für ihre Mütter
- same title
- same author(s): Claudia Dalbert
- same year (2018)
- same abstract
- different Document types: (DT=dissertation and DT2=authored book) vs only DT=dissertation
- same MT (Print)

Differences:
- DFK 0023815: the published "book" version
    - <DT>01</DT> = authored book (no other type)
    - <BNDI>Vom Fachbereich I - Psychologie an der Universität Trier 1987 als Dissertation angenommen</BNDI> -> wording indicating there is another instance in the database, and its py: 1987
    - <PU>|v Roderer |o Regensburg |i 978-3-89073-376-0</PU> -> has publisher, place and isbn!
    - <SE>Theorie und Forschung, Band 28, Psychologie, Band 12</SE> -> in a series (but not an nstitutional series, but how can we tell?)
- DFK 0025039: the (gray?) thesis version
    - <DT>61</DT> <DT2>01</DT2> - two document types: dissertation and authored book (sometimes such cases have only one type, dissertation)
    - <BN>Original als Buchausgabe erschienen: 1987</BN> -> wording that indicates there is another instance in the database, and its py: 1987
    - <PU>|v Roderer |o Regensburg |i </PU> - also has publisher and place, but no isbn - but that info is not for this instance, but the book version! Notably, isbn was omitted.
    - has no series info
    - has dissertation info: GRAD etc.
    - <DIDH>Diss.</DIDH>

## Things we don't want to merge as the same work - examples:
### A journal article and a "report", where the report is a kind of preprint of the article (is shorter)
Titles: (different punctuation, but resulting in same title_key) 
- "Verteilung des Glaubens an eine gerechte Welt in der Allgemeinbevölkerung: Normwerte für die Skala Allgemeiner Gerechte-Welt-Glaube"
- "Verteilung des Glaubens an eine gerechte Welt in der Allgemeinbevölkerung. Normwerte für die Skala Allgemeiner Gerechte-Welt-Glaube"
- same authors: 
    - Schmitt, Manfred; Dalbert, Claudia; Montada, Leo; Gschwendner, Tobias; Maes, Jürgen; Reichle, Barbara; Radant, Matthias; Schneider, Angela; Brähler, Elmar.
    - Schmitt, Manfred; Dalbert, Claudia; Montada, Leo; Gschwendner, Tobias; Maes, Jürgen; Reichle, Barbara; Radant, Matthias; Schneider, Angela; Brähler, Elmar.
- same abstract
- different years: 2007 (the report) and 2008 (the published article)
- different document types: 01 (journal article) and 02 (report)
- report version is in an institutional series that is quite common, so we might make a filter for that: "Berichte aus der Arbeitsgruppe "Verantwortung, Gerechtigkeit, Moral", Nr. 165. Trier: Universität, Fachbereich I - Psychologie, 2007, 22 S., 57 Literaturang."
- not the same identifiers: one has a handle for a repository, the other has a DOI. But preprints nowadays often have DOIs, too.
- different number of pages, thought the apparent preprint has more pages (22 vs 16), which is not what one would expect.
- the report version is marked "openAccess" in Psycharchives, once you follow the link there. Neither record has a license field, though (COPR; this was introduced ten years later). We might follow the link and get the license from there - at least if it is psycharchives. The data is in the integrated json of the psycharchives page. We might also ask an API for the license, if we have the handle or DOI.  


other works by Dalbert that might qualify:
- Belief in a just world and attitudes toward immigrants and foreign workers: A cultural comparison between Hawaii and Germany
- Belief in a just world, personality and well-being of adolescents
- Belief in a just world and adolescents' vocational and social goals
- Belief in a just world as a resource for different types of young prisoners
- Belief in a just world as a resource for unemployed young adults
- The belief in a just world and distress at school
- The "belief in a just world" construct in Hungary
- Belief in a just world and its functions for young prisoners
- Disdain of the disadvantaged: The role of responsibility denial and belief in a just world
- The importance of distinguishing the belief in a just world for self versus for others: Implications for psychological well-being
- The world is more just for me than generally: About the Personal Belief in a Just World Scale's validity
- Developmental trajectories and developmental functions of the belief in a just world. Some concluding remarks
- Parenting and young adolescents' belief in a just world
- The development of the belief in a just world. The impact of being raised in a one-parent or an intact family
- Posttraumatic symptoms, depression, and anxiety of flood victims: The impact of the belief in a just world
- Belief in a just world, well-being, and coping with an unjust fate
- The personal belief in a just world and domain-specific beliefs about justice at school and in the family: A longitudinal study with adolescents
- Personality, values and belief in a just world
- Belief in a just world as a personal resource in school
- Mental health and personality of Slovak unemployed adolescents: The impact of belief in a just world


# Think in different classes of works?
Maybe we need to differantiate the different cases where we do want to merge works:
- thesis - gray and published book
- chapters in different (reference) books - often by Springer (one of them a living reference)


When do we not want to merge works but link the works somehow, either as "has preprint", "has translation" or maybe even "has successor"?
- when one is a kind of preprint of the other:
    - do we sometimes have preprints (probably DT "report") and published versions (journal article or chapter) that seem like the same work (title, authors the same)? (e.g. in the case of theses, we should not merge as one work, but two different works linked via "has preprint")
- when one is a newer version of an article that gets published regularly (usually in the same journal), but with different data and results every time
- translations

In [6]:
# calculate and compare hashes of abstracts

import hashlib

import Levenshtein

# example abstracts from the 14 articles titled "Werbewirkungsforschung ..." which are definitely separate works:
werbewirkung_abstracts = [ 'Im Unterschied zum Werbeerfolg, der sich in ökonomischen Indizes (z. B. Umsatzveränderungen) ausdrücken lässt, stehen bei der Frage der Werbewirkung die Adressaten der werblichen Kommunikation im Mittelpunkt des Forschungsinteresses. Zum Beispiel funktioniert Werbekommunikation unterschiedlich, je nachdem, ob die Konsumenten eine Marke als "stark" oder "schwach" wahrnehmen. Daher sollten "schwache" Marken eher zurückhaltend beworben werden, um das Risiko bereits bestehender negativer Assoziationen zu vermeiden. Vor allem in neutralen oder eher langweiligen Programmumfeldern sollte darauf geachtet werden, dass Werbung "gut gemacht" ist, da sie in diesem Fall besonders auffällt und ihre Wirkung entfaltet. Dabei kann auch die häufig umstrittene erotische Werbung positive Effekte erzielen, vorausgesetzt, die Spots werden als angenehm empfunden und als gelungen beurteilt. Eine große Rolle für die Beurteilung von Produkten spielt das Vertrauen in die Presenter von Werbung, wie selbst bei menschenähnlichen Figuren, so genannten Spokes-Charakters (z.B. der "Bausparfuchs" von Wüstenrot), gezeigt werden konnte. Selbst Gestik und Körpersprache der Presenter können nicht nur emotional, sondern auch kognitiv die Wirkung der Werbebotschaft unterstützen.',
    'In einem Literaturüberblick werden aktuelle Ergebnisse der Werbewirkungsforschung referiert. Folgende Erkenntnisse werden mitgeteilt: Bei der Werbeerfolgskontrolle seien nicht nur ökonometrische Wirkungs- und Prognosemodelle, sondern auch die psychologische Erforschung der Werbewirkung von Bedeutung. So erwiesen sich ungewöhnliche, den Erwartungen der Konsumenten widersprechende Kommunikationsstrategien als sinnvoll. Beispielsweise könne man durch Normverletzungen Aufmerksamkeit steigern und die Informationsverarbeitung intensivieren. Ähnliches gelte für den Einsatz absurder Kommunikation und die Verwendung von Irritationen und Rätseln in Werbespots oder -anzeigen. Außerdem würden emotionale Inhalte besser und länger erinnert als sachliche Informationen. Die genannten Kommunikationsstrategien seien jedoch nicht per se erfolgreich, sondern erforderten eine genauere Analyse der Zielgruppe.',
    'Anhand ausgewählter empirischer Studien wird über neueste Ergebnisse der Werbewirkungsforschung informiert. Dabei wird vor allem über medien- bzw. werbepsychologisch orientierte Untersuchungen berichtet, die das komplexe Wirkungsgeflecht zwischen den spezifischen Merkmalen des Werbeangebots und den psychologischen Bedingungen auf Seiten der Rezipienten analysieren.',
    'Ein Überblick über neuere Ergebnisse wissenschaftlicher Forschung zur Werbewirkung wird gegeben. Dabei wird besonders auf Fragen der Gestaltung von Werbung und ihrer Platzierung in einem ebenfalls um Aufmerksamkeit konkurrierenden Umfeld eingegangen. Offensichtlich ist eine gute (kreative) Qualität von Werbepräsentationen eine wichtige Voraussetzung dafür, dass sie von den Rezipienten überhaupt wahrgenommen werden, das Risiko von Ablehnung verringern und positive Reaktionen der Rezipienten wahrscheinlich machen. Um Qualität zu erzielen, erscheint es erfolgversprechend, emotionale Aspekte der Gestaltung zu berücksichtigen. Positive Affekte spielen eine zentrale Rolle im Werbewirkungsprozess. Rezipienten zu involvieren und eine intensivere Verarbeitung von produktbezogenen Argumenten zu erreichen, gelingt eher mit langsamen und ruhigen Präsentationen als mit schnellen und anregenden. Die Wahrnehmung, Verarbeitung und Wirkung der Werbung hängt jedoch nicht nur von der Werbepräsentation selbst ab, sondern auch vom jeweiligen Kontext, in den sie eingebettet ist. Dazu gehören das die Werbung umgebende Programm, konkurrierende Werbung und direkt im Werbeblock positionierte Programmeigenwerbung.',
    'Neuere Studien aus der Werbewirkungsforschung werden vorgestellt. Ziel dieser Forschung ist es, den Zusammenhang zwischen Faktoren, über die der Kommunikator entscheiden kann bzw. muss (z. B. Werbedruck, Mediaplanung, Platzierung, inhaltliche und formale Gestaltung) auf der einen und rezipientenbezogenen Variablen (z. B. psychologische Befindlichkeit, Informationsverarbeitungsstrategien) auf der anderen Seite differenziert zu untersuchen, um die Wahrscheinlichkeit des Werbeerfolgs zu erhöhen. Es wird betont, dass die beschriebenen Studien den komplexen Prozess der Werbewirkung nur ausschnittsweise unter die Lupe nehmen können. Dennoch bieten sie Ansatzpunkte für die Werbepraktiker, in welcher Hinsicht der Werbeprozess optimiert werden kann, und stellen somit Entscheidungen der Kommunikatoren auf eine breitere empirische Basis.',
    'Es wird ein Überblick über einige neuere Studien gegeben, die psychologische Effekte der Werbung in Abhängigkeit von formalen und inhaltlichen Präsentations- und Gestaltungsmerkmalen untersuchen. Die Ergebnisse zeigten, dass die Einstellungen der Konsumenten gegenüber Fernsehwerbung gleichzeitig von positiven wie von negativen Aspekten geprägt sind und mit dem Ausmaß des Fernsehkonsums sowie Programmpräferenzen zusammenhängen. Weiterhin ergaben sich spezifische Wechselwirkungen zwischen Position, Spot- und Blocklänge sowie Anzahl der Konkurrenzspots innerhalb eines Werbeblocks. In einem Laborexperiment wurde festgestellt, dass eine beiläufig wahrgenommene Werbepräsentation trotz fehlender Marken-Wiederkennung die Produktwahl beeinflussen kann. Für das Ziel einer Werbepräsentation, sich beim Rezipienten durchzusetzen, sind unterschiedliche formale und inhaltliche Gestaltungsmittel bedeutsam, wie ungewöhnliche Präsentationen oder die Verwendung von Metaphern. Dabei ist jedoch darauf zu achten, dass solche Gestaltungsmerkmale die kognitiven Fähigkeiten der Rezipienten nicht zu sehr in Anspruch nehmen, um nicht vom Produkt abzulenken oder gar negative Effekte zu erzielen.',
    'Es wird im Überblick über neuere Studien aus der Werbewirkungsforschung informiert. Häufig wurde in der experimentellen Forschung die Werbebotschaft isoliert als Ursache von Wirkungen untersucht. Weit weniger Studien stellten bislang in Rechnung, dass die Rezeption von Werbung immer auch in einem Kontext, wie zum Beispiel dem redaktionellen Umfeld oder den aktuellen Stimmungslagen der Rezipienten, stattfindet. So analysierte eine Studie den Einfluss des TV-Programmumfeldes, das durchaus höhere Aufmerksamkeit gegenüber Werbung bewirken kann. Eine weitere Untersuchung fand heraus, dass der werbeumgebende Kontext offensichtlich an Bedeutung verliert, wenn die Werbung selbst intensive Reaktionen auslöst. Eine wichtige Rolle im werblichen Kommunikationsprozess spielt offensichtlich die Art und Weise, wie Rezipienten Botschaften auf sich selbst beziehen. Personen, die eher fragen "Wie geht es mir damit?" lassen sich eher durch Bilder in der Werbung ansprechen. Für Personen, die eher fragen "Was denke ich darüber?", sind verbale bzw. Textinformationen wichtiger. In diesem Zusammenhang kann sich auch Humor als effizient erweisen, er sollte sich jedoch harmonisch in die Botschaft einfügen und einen Bezug zum Produkt haben.', 
    'Neuere Studien zur Werbewirkung werden vorgestellt. Dabei wird vor allem auf folgende Forschungsthemen eingegangen: Wirksamkeit von in das laufende Fernsehprogramm eingeblendeten Werbespots, Stimmungseffekt bei Werbung, Bedeutung verschiedener, auch sprachlicher Gestaltungsvarianten von Werbung.', 
    'Im Überblick wird über neuere Studien aus der Werbewirkungsforschung informiert. Eine aktuelle Metaanalyse von Studien zur Werbeelastizität kommt zu dem Ergebnis, dass Werbung wirkt, aber in den letzten 30 Jahren immer mehr in den Werbedruck investiert werden musste, um Absatzsteigerungen zu erzielen. Werbung kann, wie weitere Studien zeigen, auch auf der vorökonomischen, das heißt psychologischen Ebene wirksam sein kann, und zwar selbst dann, wenn Konsumenten versuchen, sie zu vermeiden, vorausgesetzt, die Rezipienten hatten zumindest einen kurzen visuellen Kontakt mit der Werbebotschaft. Die intendierte Wirkung werblicher Kommunikation ist wahrscheinlicher, wenn die Konsumenten keine Signale erhalten, die sie über die persuasive Absicht der Werbekommunikation nachdenken lassen. Dies gelingt offensichtlich besser, wenn man Geschichten erzählt oder wenn die Rezipienten einen Selbstbezug zum Kommunikationsinhalt herstellen können. Auch wird die Werbewirkung deutlich dadurch beeinflusst, wie die Konsumenten sich selbst sehen und welche Aspekte des Selbstkonzepts gerade aktiviert sind. Werden beispielsweise Konsumenten durch spezifische Reize (Priming) dazu animiert, sich eher als eigenständiges Individuum denn als Teil einer Gemeinschaft zu sehen, hat dies Einfluss auf die Bewertung entsprechender werblicher Botschaften. Die beschriebenen Prozesse passieren mehr oder weniger automatisch und ohne dass bei den Rezipienten ein ausgeprägtes Bewusstsein darüber vorhanden ist. Dies gilt auch für die generelle Tendenz, Dinge zu vermenschlichen und ihnen personale Eigenschaften zuzuschreiben. Wird in Werbebotschaften eine solche "Anthropomorphisierung" durch entsprechende Kommunikationsinhalte bzw. -strategien angeregt, ergeben sich günstige Produktbewertungen.', 
    'Die Ergebnisse einiger neuerer angloamerikanischer Studien zur Werbewirkung werden vorgestellt. Die Zeiten, in denen man sich Werbewirkung als einfaches Reiz-Reaktions-Schema vorstellte, sind längst vorbei. Moderne Theorien und Wirkungsmodelle gehen davon aus, dass Werbewirkung auf unterschiedlichen Wegen zustande kommt. In einem komplexen Prozess agieren die Konsumenten aktiv und verarbeiten Informationen auf unterschiedliche Weise. Es ist zum Beispiel davon auszugehen, dass Werbewirkung nur dann entsteht, wenn die Empfänger dies zulassen. Schon die Frage, ob man sich einer Werbebotschaft zuwendet oder sie lieber vermeidet, wird durch spezifische Lebensstile und die damit verbundenen Normen und Werte beeinflusst. Befinden sich die Rezipienten in guter Stimmung, wird Werbung wohlwollender und weniger kritisch verarbeitet. Damit zusammenhängen könnte auch die Tatsache, dass kreative Werbung besser abschneidet als Durchschnittswerbung. Aus der Qualität einer Werbung ziehen die Rezipienten auch Rückschlüsse darauf, wie leistungsfähig ein werbungtreibendes Unternehmen ist und mit welcher Ernsthaftigkeit es versucht, die Verbraucher zu überzeugen.', 
    'Es wird im Überblick über neuere Studien aus der Werbewirkungsforschung informiert. Dabei wird vor allem auf die Wirksamkeit von sogenannten peripheren Merkmalen der Werbebotschaften eingegangen. Denn wenn das Involvement niedrig ist, zum Beispiel weil das angepriesene Produkt nicht sehr interessiert oder die Botschaft mit geringer Aufmerksamkeit rezipiert wird, entscheiden nicht die zentralen Argumente, sondern andere Merkmale der Botschaft, wie die Meinung der Rezipienten über das Produkt ausfällt und ob sich ein Kaufwunsch entwickelt. Zu solchen peripheren Merkmalen von Werbebotschaften gehört, von wem sie vermittelt bzw. präsentiert werden. In empirischen Studien stellte man fest, dass Prominente und andere Presenter-Typen (z. B. Experten oder typische Vertreter der Zielgruppe) für diejenigen Konsumenten bedeutsamer und damit wirkungsvoller sind, die ein starkes Bedürfnis nach sozialer Konformität haben. Vor allem bei unbekannten Marken werden den Prominenten zugeschriebene Eigenschaften auf die präsentierten Produkte übertragen. Zu den peripheren Merkmalen von Werbebotschaften zählen auch Musik und Humor. Offensichtlich ist der verwendete Musikstil ein wichtiger Faktor der Werbegestaltung. Durch Musik kann sowohl das Image des Produkts als auch das des Presenters beeinflusst werden. Humor erzielt positive Effekte auf die Beurteilung von Werbung und Produkt insbesondere bei emotionalen Produkten.', 
    '', 
    '', 
    '',]

# abstracts of two instances where we wish to extract the work, because it is the same:
interkult_abstracts = ['Die zunehmende Forderung nach der Entwicklung Interkultureller Kompetenz führt zu der Frage, was genau unter diesem Konstrukt verstanden wird, wie Interkulturelle Kompetenz erfasst werden kann und welche Möglichkeiten der Förderung bestehen. Aufgrund dessen wird das Konstrukt der Interkulturellen Kompetenz zunächst kritisch betrachtet. Es zeigt sich, dass bisher kein allgemein anerkanntes, theoretisch fundiertes und empirisch bewährtes Gesamtmodell zur Interkulturellen Kompetenz entwickelt werden konnte. Des Weiteren verfügen die bestehenden Modelle und Definitionen über eine zu geringe Trennschärfe zu anderen Konstrukten wie beispielsweise die Kulturelle Intelligenz und auch die vorhandenen Messinstrumente verfügen nur über unzureichende Gütekriterien. Die Betrachtung der Einsatzgebiete Interkultureller Kompetenz anhand von Fallbeispielen in den Bereichen Bildung, öffentliche Verwaltung und Wirtschaft verdeutlichen den Bedarf an adäquaten Definitionen und zuverlässigen Methoden der Messung und Förderung. In der Forschung und Praxis zeigt sich, dass statt an wissenschaftlich basierten Instrumentarien zur Messung und Entwicklung Interkultureller Kompetenzen an einer Überbetonung kulturspezifischen Wissens festgehalten wird und mit welchem die Handlungsfähigkeit in interkulturellen Überschneidungssituationen aufrechterhalten werden soll. Die klare Definition, Messbarkeit und Trainierbarkeit stellt somit nicht ausschließlich ein Ziel für die Wissenschaft dar, sondern auch für die Professionalisierung der Kompetenzentwicklung in Bildung und Personalentwicklung. (c) Springer Fachmedien Wiesbaden GmbH', 
'Die zunehmende Forderung nach der Entwicklung Interkultureller Kompetenz führt zu der Frage, was genau unter diesem Konstrukt verstanden wird, wie Interkulturelle Kompetenz erfasst werden kann und welche Möglichkeiten der Förderung bestehen. Aufgrund dessen wird das Konstrukt der Interkulturellen Kompetenz zunächst kritisch betrachtet. Es zeigt sich, dass bisher kein allgemein anerkanntes, theoretisch fundiertes und empirisch bewährtes Gesamtmodell zur Interkulturellen Kompetenz entwickelt werden konnte. Des Weiteren verfügen die bestehenden Modelle und Definitionen über eine zu geringe Trennschärfe zu anderen Konstrukten wie beispielsweise die Kulturelle Intelligenz und auch die vorhandenen Messinstrumente verfügen nur über unzureichende Gütekriterien. Die Betrachtung der Einsatzgebiete Interkultureller Kompetenz anhand von Fallbeispielen in den Bereichen Bildung, öffentliche Verwaltung und Wirtschaft verdeutlichen den Bedarf an adäquaten Definitionen und zuverlässigen Methoden der Messung und Förderung. In der Forschung und Praxis zeigt sich, dass statt an wissenschaftlich basierten Instrumentarien zur Messung und Entwicklung Interkultureller Kompetenzen an einer Überbetonung kulturspezifischen Wissens festgehalten wird und mit welchem die Handlungsfähigkeit in interkulturellen Überschneidungssituationen aufrechterhalten werden soll. Die klare Definition, Messbarkeit und Trainierbarkeit stellt somit nicht ausschließlich ein Ziel für die Wissenschaft dar, sondern auch für die Professionalisierung der Kompetenzentwicklung in Bildung und Personalentwicklung. (c) Springer Fachmedien Wiesbaden GmbH'
]

# calculate hash for each abstract:
hashes = []
for abstract in werbewirkung_abstracts:
    hashes.append(hashlib.md5(abstract.encode('utf-8')).hexdigest())

# print hashes:
print("== hashes for werbewirkung abstracts: ==")
# -> a visual check shows: all but the last 3 abstracts are unique - and those 3 are empty strings, because the abstracts are not available.
# we can use this to identify works that must not be merged, but only in combination with another heuristic: they are also in the same journal (JT is the same or a fuzzy match) and, possibly, have a different publication year. 
for hash in hashes:
    print(hash)


hashes_interkult = []
for abstract in interkult_abstracts:
    hashes_interkult.append(hashlib.md5(abstract.encode('utf-8')).hexdigest())

# print hashes:
# spoiler: they are the same! Which can help us determine that it is the same work! However, sometimes they are the same, but it is not the same work, but one is a preprint (DT=report) and the other is the published version.
print("\n== hashes for interkult abstracts: ==")
for hash in hashes_interkult:
    print(hash)
    
# calculate similarity matrix:
# similarity_matrix = []
# for i in range(len(hashes)):
#     similarity_matrix.append([])
#     for j in range(len(hashes)):
#         similarity_matrix[i].append(Levenshtein(hashes[i], hashes[j]))

# # print similarity matrix:
# for row in similarity_matrix:
#     print(row)

# print similarity matrix as heatmap:
# plt.imshow(similarity_matrix, cmap='hot', interpolation='nearest')
# plt.show()

# # print similarity matrix as heatmap with labels:
# fig, ax = plt.subplots()
# im = ax.imshow(similarity_matrix, cmap='hot', interpolation='nearest')

# # We want to show all ticks...
# ax.set_xticks(np.arange(len(abstracts)))
# ax.set_yticks(np.arange(len(abstracts)))
# # ... and label them with the respective list entries
# ax.set_xticklabels(abstracts)
# ax.set_yticklabels(abstracts)




== hashes for werbewirkung abstracts: ==
ca89d866699a396194a379604d921eab
d5367a93767d2c8e16595799cd6eff5e
54aa7f76d4ef4baad4b3c7e05e39bdfe
6c4314842a2fd056d2229eb73734ca86
63cafced23726e8620204b77a9b9ea2b
7900998308d77f7dc6dc19a8d4e34f4a
e84d579f3f2ad83c75a70419546bbed5
9c8c050b08d91090544858e54698cf3a
c7d220e0177400883f69bf136f536ef3
691d1d1f47340565c0ae9a39d4e5d331
4e466b23d97a614e3cef543dcfd4cacf
d41d8cd98f00b204e9800998ecf8427e
d41d8cd98f00b204e9800998ecf8427e
d41d8cd98f00b204e9800998ecf8427e

== hashes for interkult abstracts: ==
7439b44182d58e2ce5eb2b92723a2f2e
7439b44182d58e2ce5eb2b92723a2f2e
